In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
from pandarallel import pandarallel
from pathlib import Path
from transformers import AutoTokenizer
from transformers import BertForSequenceClassification
from torch.optim import Adam
from pandarallel import pandarallel
import torch
from tqdm.auto import tqdm, trange
from transformers import TrainingArguments, Trainer
import pandas as pd


def get_label2id_id2label(class_labels):
    id2label = dict(enumerate(class_labels))
    label2id = {i: label for label, i in id2label.items()}
    return label2id, id2label


def push_to_hub(model, tokenizer, name):
    model.push_to_hub(name)
    tokenizer.push_to_hub(name)


RANDOM_STATE = 42
PANDARALLEL_WORKERS = 8
labels = [
    "Россия",
    "Экономика",
    "Силовые структуры",
    "Бывший СССР",
    "Спорт",
    "Забота о себе",
    "Строительство",
    "Путешествия",
    "Наука и техника",
]
columns = ["text", "label"]
news_path = "data/news/lenta-ru-news.csv.bz2"

tokenizer = model = "cointegrated/rubert-tiny2"

tokenizer = AutoTokenizer.from_pretrained(model)

label2id, id2label = get_label2id_id2label(labels)
model = BertForSequenceClassification.from_pretrained(
    model,
    num_labels=len(labels),
    problem_type="single_label_classification",
    label2id=label2id,
    id2label=id2label,
)


max_length = 256
batch_size = 32
num_workers = 4
num_epochs = 3

print(label2id)

/home/seara/.pyenv/versions/3.9.16/envs/DS/lib/python3.9/site-packages/pydantic/_internal/_fields.py:128: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/seara/.pyenv/versions/3.9.16/envs/DS/lib/python3.9/site-packages/pydantic/_internal/_config.py:317: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'Россия': 0, 'Экономика': 1, 'Силовые структуры': 2, 'Бывший СССР': 3, 'Спорт': 4, 'Забота о себе': 5, 'Строительство': 6, 'Путешествия': 7, 'Наука и техника': 8}


In [2]:
# df[(df["topic"] == "Среда обитания") & (df["tags"] == "Дом")].sample(5)


def clean_text(text):
    text = text.replace("\n", " ")
    text = text.replace("\t", "")
    text = text.replace("\\xa0", " ")
    text = text.split()
    text = " ".join(text)
    return text


def get_fontanka():
    datasets = []
    for year in [2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]:
        datasets.append(
            pd.read_csv(f"data/news/Fontanka/metatable_{year}.csv", sep="\t")
        )

    def format_labels(text):
        # if text in ["Финансы", "Бизнес"]:
        #     return "Экономика"
        # if text == "Технологии":
        #     return "Наука и техника"
        if text == "Туризм":
            return "Путешествия"
        if text == "Общество":
            return "Россия"
        return text

    def read_text(text_id):
        year = str(text_id)[:4]
        return open(f"data/news/Fontanka/texts/{year}/fontanka_{text_id}.txt").read()

    fontanka = pd.concat(datasets)
    fontanka = fontanka[
        fontanka["textrubric"].isin(
            [
                "Общество",
                # "Финансы",
                # "Проишествия",
                "Спорт",
                "Строительство",
                # "Бизнес",
                "Туризм",
                # "Технологии",
            ]
        )
    ]
    fontanka = fontanka[["textid", "textrubric"]]
    fontanka["textrubric"] = fontanka["textrubric"].apply(format_labels)
    fontanka["textid"] = fontanka["textid"].astype(int)
    fontanka["textid"] = fontanka["textid"].apply(read_text)
    fontanka.columns = ["text", "label"]
    fontanka["label"] = fontanka["label"].apply(lambda x: label2id[x])
    fontanka["text"] = fontanka["text"].apply(clean_text)
    fontanka = fontanka.dropna(subset="text")
    return fontanka


dataset_path = Path("data", "news", "news.csv")
# if dataset_path.is_file():
#     final = pd.read_csv(dataset_path)
# else:
topics = [
    "Россия",
    "Экономика",
    "Силовые структуры",
    "Бывший СССР",
    "Спорт",
    "Забота о себе",
    # "Среда обитания",
    "Путешествия",
    "Наука и техника",
    # "Дом",
]
df = pd.read_csv(news_path)
df = df.dropna(subset="text")
df = df[df["topic"].isin(topics)][["text", "topic"]]
# df["topic"] = df["topic"].apply(
#     lambda x: "Строительство" if x in ["Среда обитания", "Дом"] else x
# )
df["text"] = df["text"].apply(clean_text)
df.columns = ["text", "label"]
df["label"] = df["label"].apply(lambda x: label2id[x])

# dataset_2 = pd.read_csv("data/news/FULL_data_tokenized_v3.csv")
# dataset_2 = dataset_2.drop("processed", axis=1)
# dataset_2.columns = ["text", "label"]
# dataset_2["text"] = dataset_2["text"].apply(clean_text)

fontanka = get_fontanka()

final = pd.concat([df, fontanka])
# final.to_csv(dataset_path, index=False)

In [3]:
final.isna().sum()

text     0
label    0
dtype: int64

In [ ]:
trash_words = ["Фото:", "Кадр:", "PAN Photo", "Изображение:", "Поделиться"]
sources = [
    "Globallookpress.com",
    "РИА Новости",
    "«Коммерсантъ»",
    "Коммерсант",
    "Коммерсантъ",
    "Reuter",
    "Reuters",
    "Unsplash",
    "Getty Images",
    "Global Look Press",
    "Depositphotos",
    "Агентство «Москва»",
    "ТАСС",
    "Wikipedia",
    "«Фонтанка.ру»Поделиться",
    "«Вестас Рус»",
    "Depositphotos",
    "Unsplash.com",
    "РИА Новости",
    "YouTubeПоделиться",
    "Shutterstock",
    "president.gov.by",
    "Pixabay",
    "YouTube",
    "pixabay.com",
    "«Лента.ру»",
    "life.ru",
    "Boards.4channel.org",
    "страница Lifedd.ru |",
    "E1.RUПоделиться",
    "JAXA",
    "news.ru via globallookpress.co",
    "Wikimedia Commons",
    "Fotodom",
    "Bloomberg via Getty Images",
    "Ukran-orosz konfliktus",
    "Telegram",
    "«Фонтанка.ру»",
    "-канал SHOT",
]
trash_words.sort(key=len, reverse=True)
sources.sort(key=len, reverse=True)


def remove_slash_prefix(text: str):
    text = text.split()
    start = 0
    for i in range(len(text)):
        if text[i] in ["/", "\\"] and i <= 6:
            start = i
    if start != 0:
        start += 1
    return " ".join(text[start:])


def remove_extra_whitespaces(text: str):
    text = text.split()
    return " ".join(text)


def remove_trash_words(text: str):
    for word in trash_words:
        text = text.removeprefix(word)
        text = text.removeprefix(word.lower())
        text = text.removeprefix(word.upper())

    for word in sources:
        text = text.removeprefix(word)
        text = text.removeprefix(word.lower())
        text = text.removeprefix(word.upper())

    return text


def preprocess_text(text: str):
    text = (
        text.replace("\n", " ")
        .replace("\t", "")
        .replace("/", " / ")
        .replace("\\", " \\ ")
    )
    text = remove_extra_whitespaces(text)

    text = remove_trash_words(text)

    text = remove_slash_prefix(text)
    text = remove_slash_prefix(text)

    text = remove_trash_words(text)
    text = remove_trash_words(text)

    text = remove_extra_whitespaces(text)
    return text

test = pd.read_csv("test_news.csv")
test["content"] = test["content"].apply(preprocess_text)
test

In [13]:
train_dataset = Dataset.from_pandas(final, preserve_index=False)

train_processed = train_dataset.map(
    lambda x: tokenizer(x["text"], truncation=True, max_length=max_length),
    batched=True,
    remove_columns=["text"],
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_dataloader = DataLoader(
    train_processed,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    collate_fn=data_collator,
    pin_memory=False,
    drop_last=False,
)

Map:   0%|          | 0/786966 [00:00<?, ? examples/s]

In [ ]:
test_dataset = Dataset.from_pandas(test, preserve_index=False)

test_processed = test_dataset.map(
    lambda x: tokenizer(x['content'], truncation=True, max_length=max_length),
    batched=True,
    remove_columns=['content'],
)

test_dataloader = DataLoader(
    test_processed,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    collate_fn=data_collator,
    pin_memory=False,
    drop_last=False,
)

In [14]:
model.cuda()
optimizer = Adam(params=model.parameters(), lr=0.00002, weight_decay=0)

for epoch in trange(num_epochs):
    
    model.train()
    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()
        batch = batch.to(model.device)
        output = model(**batch)
        loss = output.loss
        loss.backward()
        optimizer.step()
        
    tqdm.write(f"====================={loss.item()}=====================")
    
    model.eval()
    y_pred = []
    with torch.inference_mode():
        for batch in tqdm(test_dataloader):
            batch = batch.to(model.device)
            output = model(**batch)
            y_pred.append(output.logits.cpu())

    preds = torch.softmax(torch.cat(y_pred), dim=-1)

    sub = test.reset_index()
    sub["topic"] = torch.argmax(preds, dim=-1).numpy()
    sub[["topic", "index"]].to_csv(f"bs-{batch_size}-ep-{num_epochs}-clean-topics.csv", index=False)
    

  0%|          | 0/1 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

  0%|          | 0/24593 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


0.013102412223815918


In [34]:
# push_to_hub(model, tokenizer, "seara/merged-1-epoch")

pytorch_model.bin:   0%|          | 0.00/117M [00:00<?, ?B/s]

In [15]:
trash_words = ["Фото:", "Кадр:", "PAN Photo", "Изображение:", "Поделиться"]
sources = [
    "Globallookpress.com",
    "РИА Новости",
    "«Коммерсантъ»",
    "Коммерсант",
    "Коммерсантъ",
    "Reuter",
    "Reuters",
    "Unsplash",
    "Getty Images",
    "Global Look Press",
    "Depositphotos",
    "Агентство «Москва»",
    "ТАСС",
    "Wikipedia",
    "«Фонтанка.ру»Поделиться",
    "«Вестас Рус»",
    "Depositphotos",
    "Unsplash.com",
    "РИА Новости",
    "YouTubeПоделиться",
    "Shutterstock",
    "president.gov.by",
    "Pixabay",
    "YouTube",
    "pixabay.com",
    "«Лента.ру»",
    "life.ru",
    "Boards.4channel.org",
    "страница Lifedd.ru |",
    "E1.RUПоделиться",
    "JAXA",
    "news.ru via globallookpress.co",
    "Wikimedia Commons",
    "Fotodom",
    "Bloomberg via Getty Images",
    "Ukran-orosz konfliktus",
    "Telegram",
    "«Фонтанка.ру»",
    "-канал SHOT",
]
trash_words.sort(key=len, reverse=True)
sources.sort(key=len, reverse=True)


def remove_slash_prefix(text: str):
    text = text.split()
    start = 0
    for i in range(len(text)):
        if text[i] in ["/", "\\"] and i <= 6:
            start = i
    if start != 0:
        start += 1
    return " ".join(text[start:])


def remove_extra_whitespaces(text: str):
    text = text.split()
    return " ".join(text)


def remove_trash_words(text: str):
    for word in trash_words:
        text = text.removeprefix(word)
        text = text.removeprefix(word.lower())
        text = text.removeprefix(word.upper())

    for word in sources:
        text = text.removeprefix(word)
        text = text.removeprefix(word.lower())
        text = text.removeprefix(word.upper())

    return text


def preprocess_text(text: str):
    text = (
        text.replace("\n", " ")
        .replace("\t", "")
        .replace("/", " / ")
        .replace("\\", " \\ ")
    )
    text = remove_extra_whitespaces(text)

    text = remove_trash_words(text)

    text = remove_slash_prefix(text)
    text = remove_slash_prefix(text)

    text = remove_trash_words(text)
    text = remove_trash_words(text)

    text = remove_extra_whitespaces(text)
    return text

test = pd.read_csv("test_news.csv")
test["content"] = test["content"].apply(preprocess_text)
test


test_dataset = Dataset.from_pandas(test, preserve_index=False)

test_processed = test_dataset.map(
    lambda x: tokenizer(x['content'], truncation=True, max_length=max_length),
    batched=True,
    remove_columns=['content'],
)

test_dataloader = DataLoader(
    test_processed,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    collate_fn=data_collator,
    pin_memory=False,
    drop_last=False,
)



model.eval()
y_pred = []
with torch.inference_mode():
    for batch in tqdm(test_dataloader):
        batch = batch.to(model.device)
        output = model(**batch)
        y_pred.append(output.logits.cpu())

preds = torch.softmax(torch.cat(y_pred), dim=-1)

sub = test.reset_index()
sub["topic"] = torch.argmax(preds, dim=-1).numpy()
sub[["topic", "index"]].to_csv(f"bs-{batch_size}-ep-{num_epochs}-clean-topics.csv", index=False)

Map:   0%|          | 0/26275 [00:00<?, ? examples/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/822 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


,content
0,Экс-министру обороны ДНР Игорю Стрелкову (Гирк...
1,В начале февраля 2023 года в Пушкинском районе...
2,Анастасия Борисова Международная федерация спо...
3,"Если вы хотели, но так и не съездили на море л..."
4,Российский полузащитник «Чертаново» Сергей Пин...
...,...
26270,Алевтина Запольская Главное управление разведк...
26271,Wikimedia Алевтина Запольская Министр молодежи...
26272,Александр Курбатов Октябрьский районный суд го...
26273,Варвара Кошечкина Президент Украины Владимир З...


Map:   0%|          | 0/26275 [00:00<?, ? examples/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/822 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
push_to_hub(model, tokenizer, "clean-topics")

pytorch_model.bin:   0%|          | 0.00/117M [00:00<?, ?B/s]

In [1]:
from transformers import pipeline

model = pipeline(model="seara/merged-1-epoch", device="cuda")

/home/seara/.pyenv/versions/3.9.16/envs/DS/lib/python3.9/site-packages/pydantic/_internal/_fields.py:128: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/seara/.pyenv/versions/3.9.16/envs/DS/lib/python3.9/site-packages/pydantic/_internal/_config.py:317: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [4]:
from src.utils.utils import get_label2id_id2label

topics = [
    "Россия",
    "Экономика",
    "Силовые структуры",
    "Бывший СССР",
    "Спорт",
    "Забота о себе",
    "Строительство",
    "Путешествия",
    "Наука и техника",
]
a, _ = get_label2id_id2label(topics)


ans = model(
    test["content"].values.tolist(), batch_size=64, truncation=True, max_length=256
)
out = []
for item in ans:
    out.append(a[item["label"]])

In [5]:
sub = test.reset_index()
sub["topic"] = out
sub[["topic", "index"]].to_csv("submission3.csv", index=False)

In [41]:
out

[0,
 6,
 4,
 0,
 4,
 3,
 2,
 3,
 0,
 3,
 0,
 1,
 1,
 1,
 5,
 4,
 0,
 0,
 0,
 0,
 1,
 8,
 6,
 0,
 1,
 1,
 0,
 0,
 3,
 8,
 0,
 6,
 0,
 1,
 1,
 1,
 1,
 1,
 8,
 0,
 7,
 4,
 0,
 0,
 3,
 0,
 0,
 8,
 0,
 0,
 3,
 3,
 0,
 1,
 0,
 3,
 4,
 0,
 2,
 2,
 1,
 1,
 0,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 0,
 1,
 3,
 0,
 0,
 7,
 8,
 0,
 0,
 4,
 8,
 0,
 0,
 0,
 1,
 1,
 8,
 0,
 3,
 0,
 0,
 0,
 3,
 0,
 3,
 0,
 0,
 4,
 0,
 3,
 0,
 0,
 3,
 3,
 5,
 0,
 1,
 3,
 0,
 2,
 0,
 0,
 1,
 3,
 0,
 1,
 4,
 1,
 4,
 7,
 7,
 0,
 0,
 0,
 6,
 3,
 0,
 3,
 1,
 4,
 4,
 0,
 0,
 0,
 1,
 0,
 7,
 0,
 1,
 0,
 0,
 0,
 0,
 4,
 0,
 3,
 3,
 3,
 4,
 0,
 0,
 1,
 0,
 1,
 0,
 3,
 2,
 1,
 2,
 4,
 0,
 0,
 0,
 3,
 0,
 0,
 3,
 0,
 0,
 0,
 6,
 0,
 1,
 0,
 0,
 3,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 4,
 2,
 0,
 0,
 8,
 3,
 1,
 0,
 4,
 3,
 8,
 0,
 3,
 0,
 6,
 0,
 8,
 1,
 0,
 2,
 7,
 1,
 1,
 3,
 0,
 1,
 5,
 4,
 1,
 0,
 3,
 3,
 0,
 2,
 3,
 3,
 2,
 0,
 0,
 4,
 4,
 3,
 5,
 1,
 1,
 7,
 0,
 6,
 0,
 0,
 4,
 6,
 1,
 3,
 0,
 0,
 0,
 6,
 7,
 7,
 0,
 0,
 3,
 0,
 4,


In [40]:
sub

,index,content,topic
0,0,Фото: «Фонтанка.ру»ПоделитьсяЭкс-министру обор...,0
1,1,В начале февраля 2023 года в Пушкинском районе...,6
2,2,Фото: Andy Bao / Getty Images Анастасия Борисо...,4
3,3,"Если вы хотели, но так и не съездили на море л...",0
4,4,Сергей Пиняев Фото: Алексей Филиппов / РИА Нов...,4
...,...,...,...
26270,26270,Фото: РИА Новости Алевтина Запольская Главное ...,3
26271,26271,Вадим Гутцайт Фото: Sergei CHUZAVKOV / Europea...,3
26272,26272,Фото: Олег Харсеев / Коммерсантъ Александр Кур...,0
26273,26273,Владимир Зеленский Фото: Yves Herman / Reuters...,3


In [37]:
pd.set_option("max_colwidth", None)
sub[sub["topic"] == 7]

,index,content,topic
40,40,"Дарья Рыбак Россиянка отдохнула в «райском уголке» Краснодарского края под Геленджиком и осталась разочарована поездкой. Отзывом о своем путешествии она поделилась в своем блоге на платформе «Яндекс.Дзен». Туристка уточнила, что всегда считала поселок Голубая бухта, который находится в 10 километрах от Геленджика, «райским уголком». По ее воспоминаниям, место привлекало покоем и тишиной, а также славилось голубой водой и дельфинами, «которые подплывали к пирсу». Тем не менее, последняя поездка в это место разочаровала п его, женщина заметила, что «море не такое голубое как раньше» — цвет воды показался ей «мутным и серым». Из плюсов автор назвала недорогую стоимость жилья — так, большой номер в 15 минутах ходьбы от пляжа обходился ей в 1,2 тысячи рублей в сутки. Ранее в июле эта же путешественница бюджетно отдохнула в поселке Ольгинка в Краснодарском крае и также была разочарована чистотой и инфраструктурой пляжа. По словам автора, приехать на курорт ей посоветовали знакомые, которые были в этом месте несколько лет назад, однако место «полностью разочаровало» россиянку.",7
75,75,"Мария Гейн Россиянка застряла в Таиланде почти на месяц и рассказала о попытках вернуться домой. Своим опытом она поделилась с порталом «Тонкости туризма». По словам девушки, как только начала поступать информация о массовой отмене рейсов, она решила переждать ограничения. «Первые несколько дней была растерянность, новости были одна страшнее другой», — призналась она. Также путешествен автор успела на один из последних рейсов Air Astana и добралась до России с пересадкой в Казахстане. Ранее в марте сообщалось, что жители Таиланда начали помогать застрявшим в стране российским туристам. Так, они организовали центры помощи и горячие линии для семи тысяч путешественников из России. Отдыхающим, у которых закончился период безвизового пребывания, продлили возможность нахождения в королевстве на 30 суток.",7
119,119,"Россия и Турция договорились увеличить число авиаперевозок в летнем сезоне. Российские компании смогут выполнять более 720 рейсов по 62 маршрутам, турецкие — 629 рейсов по 37 маршрутам, сообщает «Интерфакс» со ссылкой на Минтранс.Российские авиационные власти уже подтвердили заявки турецких авиакомпаний на дополнительные полеты с суммарной частотой 49 рейсов в неделю. Так, число еженедельно выполняемых рейсов между странами превысит 1,3 тысячи, а это в 1,5 раза больше по сравнению с предыдущим летним сезоном. Согласно договоренностям, еженедельно по маршруту Анталья — Саратов будет совершаться семь рейсов, по маршруту Стамбул — Москва — Анталья — Москва — по 21 ре а кататься по морю на доске, но внезапно усилился ветер, который стал уносить людей от берега. Выбраться отдыхающим помогли другие туристы, которые катались в том же месте на катамаране, — они подтолкнули доску к побережью, где волны были более спокойными. «Женщина с ребенком поплыла, а ветер сильный. (...) И их стало уносить в море, она испугалась, стала орать. Мы, конечно же, пришли на помощь», — рассказала автор видео, отметив, что операция прошла благополучно. Ранее в сети появилось видео, на котором отдыхающие пытались «всем пляжем» спасти женщину, которой стало плохо в море у берега в Сочи. Туристка скончалась на побережье российского курорта до прибытия врачей.",7
120,120,"Лука Сафронов - Затравкин Фото: @lukasafronovofficial Российский пианист, сын художника Никаса Сафронова, Лука Затравкин раскрыл правду о появившейся в СМИ информации по поводу его заключения в турецком отеле из-за поломанного бассейна. Его слова приводит «Пятый канал». Как рассказал Затравкин, сотрудники гостиницы действительно попросили его компенсировать ремонт бассейна, дно которого было пробито в результате неудачного прыжка музыканта. Тем не менее, по его заверениям, никто не удерживает его в п за ремонт сломанного им бассейна. Отец пианиста сообщил, что музыкант не может вернуться в Россию из-за того, что сотрудники гостиницы заставляют его компенсировать ремонт конструкции 

In [2]:
# class_ = 8
# sub = df.reset_index()
# sub = sub.drop("content", axis=1)
# sub["topic"] = class_
# sub[["topic", "index"]].to_csv(f"{class_}.csv", index=False)

length_full = 26275
classes_distribution = {
    0: 0.43393,
    1: 0.11945,
    2: 0.07057,
    3: 0.12451,
    4: 0.0965,
    5: 0.0137,
    6: 0.02778,
    7: 0.05116,
    8: 0.06236,
}
distributions = pd.DataFrame([classes_distribution]).T

In [3]:
sub = pd.read_csv("3-epoch-clean-topics.csv")

In [4]:
sub

,topic,index
0,0,0
1,6,1
2,4,2
3,0,3
4,4,4
...,...,...
26270,3,26270
26271,3,26271
26272,3,26272
26273,3,26273


In [5]:
comparison = pd.concat(
    [sub["topic"].value_counts(), (distributions[0] * length_full).astype(int)], axis=1
)
comparison.columns = ["pred", "true"]
comparison["diff"] = (comparison["pred"] - comparison["true"]).abs()
comparison.sort_index()

,pred,true,diff
0,11032,11401,369
1,3430,3138,292
2,1791,1854,63
3,3062,3271,209
4,2619,2535,84
5,372,359,13
6,747,729,18
7,1490,1344,146
8,1732,1638,94


In [6]:
print(f"Max acc: {1- comparison['diff'].sum() / length_full}")

Max acc: 0.9509800190294957


In [22]:
comparison = pd.concat(
    [sub["topic"].value_counts(), (distributions[0] * length_full).astype(int)], axis=1
)
comparison.columns = ["pred", "true"]
comparison["diff"] = (comparison["pred"] - comparison["true"]).abs()
comparison.sort_index()

,pred,true,diff
0,10120,11401,1281
1,3891,3138,753
2,2064,1854,210
3,3414,3271,143
4,2609,2535,74
5,395,359,36
6,501,729,228
7,1483,1344,139
8,1798,1638,160


In [19]:
print(f"Max acc: {1- comparison['diff'].sum() / length_full}")

Max acc: 0.8849096098953377


In [ ]:
# def splitter(text):
#     text = text.split()
#     if len(text) > 80:
#         text = text[40:]
#     return " ".join(text)
# df["content1"] = df["content"].apply(splitter)
# df["content2"] = df["content1"].apply(splitter)
# df["content3"] = df["content2"].apply(splitter)
# df["content4"] = df["content3"].apply(splitter)
# df["content5"] = df["content4"].apply(splitter)

# from src.utils.utils import get_label2id_id2label

# topics = [
#     "Россия",
#     "Экономика",
#     "Силовые структуры",
#     "Бывший СССР",
#     "Спорт",
#     "Забота о себе",
#     "Строительство",
#     "Путешествия",
#     "Наука и техника",
# ]
# a, _ = get_label2id_id2label(topics)

# final = {}
# for column in ["content", "content1", "content2", "content3", "content4", "content5"]:
#     print(column)
#     ans = model(
#         df[column].values.tolist(), batch_size=64, truncation=True, max_length=256
#     )
#     out = []
#     for item in ans:
#         out.append(a[item["label"]])
#     final[column] = out